In [1]:
import nltk
import re
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
%matplotlib inline

In [2]:
df=pd.read_csv(r"G:\m.tech 2ndsem\nlp\absa_refined_latest.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'Clothing ID', 'Age', 'Rating', 'Review', 'Division Name',
       'aspect_keywords', 'sentiment', 'Department Name', 'Class Name'],
      dtype='object')

In [4]:
df1=df.iloc[:,4:8]

In [5]:
df1

,Review,Division Name,aspect_keywords,sentiment
0,Absolutely wonderful - silky and sexy and com...,Initmates,[],pos
1,Love this dress! it's sooo pretty. i happen...,General,"['never ordered', 'ordered online', 'true midi']",pos
2,Some major design flawsI had such high hopes f...,General,"['major design', 'initially ordered', 'such ho...",pos
3,"My favorite buy!I love, love, love this jumpsu...",General Petite,"['favorite buy!I love', 'great compliments']",pos
4,Flattering shirtThis shirt is very flattering ...,General,"['adjustable front tie', 'Flattering shirt', '...",pos
...,...,...,...,...
22623,Great dress for many occasionsI was very happy...,General Petite,"['great price', 'easy to slip', 'Great dress',...",pos
22624,Wish it was made of cottonIt reminds me of mat...,General Petite,"['long sleeves', 'soft material', 'nice sleeve...",pos
22625,"Cute, but see throughThis fit well, but the to...",General Petite,"['different fabric', 'fit well', 'never worked...",pos
22626,"Very cute dress, perfect for summer parties an...",General,"['much cheaper t', 'cheaper t', 'cute dress', ...",pos


In [6]:
df1=df1.drop(['Division Name','aspect_keywords'],axis=1)

In [7]:
df1

,Review,sentiment
0,Absolutely wonderful - silky and sexy and com...,pos
1,Love this dress! it's sooo pretty. i happen...,pos
2,Some major design flawsI had such high hopes f...,pos
3,"My favorite buy!I love, love, love this jumpsu...",pos
4,Flattering shirtThis shirt is very flattering ...,pos
...,...,...
22623,Great dress for many occasionsI was very happy...,pos
22624,Wish it was made of cottonIt reminds me of mat...,pos
22625,"Cute, but see throughThis fit well, but the to...",pos
22626,"Very cute dress, perfect for summer parties an...",pos


In [20]:
X = df.Review
Y = df.sentiment
le = LabelEncoder()
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [21]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [22]:
max_words = 1000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [31]:
def lstm():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer =  Bidirectional(LSTM(64))(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [24]:
model = lstm()
model.summary()
model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               58880     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               33024     
_________________________________________________________________
activation_2 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [25]:
model.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 15386 samples, validate on 3847 samples
Epoch 1/10
15386/15386 [==============================] - 168s 11ms/sample - loss: 0.2720 - accuracy: 0.9230 - val_loss: 0.2418 - val_accuracy: 0.9288
Epoch 2/10
15386/15386 [==============================] - 146s 9ms/sample - loss: 0.2287 - accuracy: 0.9294 - val_loss: 0.2379 - val_accuracy: 0.9290
Epoch 3/10
15386/15386 [==============================] - 140s 9ms/sample - loss: 0.2127 - accuracy: 0.9305 - val_loss: 0.2467 - val_accuracy: 0.9296


In [26]:
test_sequences = tok.texts_to_sequences(X_test)
test_sequences_matrix = sequence.pad_sequences(test_sequences,maxlen=max_len)

In [28]:
print('Test set\n     Accuracy: {:0.3f}'.format(accr[1]))

Test set
     Accuracy: 0.929


In [29]:
sequences_matrix[1]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,  17,   6, 405,   5,   6,  23, 263,   3,
       141, 214,  11,   1,  45, 223,   5,  60,  30,   4,  48,  17,   1,
       277,   6, 150,   3,   1, 340,  29,  26, 546, 714,  12, 820,   4,
        23,  86,  17, 482, 431,   3, 151])

In [30]:
df1.loc[1]

Review        Love this dress!  it's sooo pretty.  i happen...
sentiment                                                  pos
Name: 1, dtype: object

In [32]:
def lstm1():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(50)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [33]:
model1 = lstm1()
model1.summary()
model1.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 150, 50)           50000     
_________________________________________________________________
lstm_2 (LSTM)                (None, 50)                20200     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               13056     
_________________________________________________________________
activation_4 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 1)                 257 

In [34]:
model1.fit(sequences_matrix,Y_train,batch_size=128,epochs=10,
          validation_split=0.2,callbacks=[EarlyStopping(monitor='val_loss',min_delta=0.0001)])

Train on 15386 samples, validate on 3847 samples
Epoch 1/10
15386/15386 [==============================] - 73s 5ms/sample - loss: 0.2713 - accuracy: 0.9245 - val_loss: 0.2577 - val_accuracy: 0.9288
Epoch 2/10
15386/15386 [==============================] - 62s 4ms/sample - loss: 0.2250 - accuracy: 0.9291 - val_loss: 0.2588 - val_accuracy: 0.9280


In [35]:
test_sequences1 = tok.texts_to_sequences(X_test)
test_sequences_matrix1 = sequence.pad_sequences(test_sequences1,maxlen=max_len)

In [36]:
accr1 = model1.evaluate(test_sequences_matrix1,Y_test)

3395/1 [================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================

In [37]:
print('Test set\n     Accuracy: {:0.3f}'.format(accr1[1]))

Test set
     Accuracy: 0.927


In [3]:
df

,Unnamed: 0,Clothing ID,Age,Rating,Review,Division Name,aspect_keywords,sentiment,Department Name,Class Name
0,0,767,33,4,Absolutely wonderful - silky and sexy and com...,Initmates,[],pos,Intimate,Intimates
1,1,1080,34,5,Love this dress! it's sooo pretty. i happen...,General,"['never ordered', 'ordered online', 'true midi']",pos,Dresses,Dresses
2,2,1077,60,3,Some major design flawsI had such high hopes f...,General,"['major design', 'initially ordered', 'such ho...",pos,Dresses,Dresses
3,3,1049,50,5,"My favorite buy!I love, love, love this jumpsu...",General Petite,"['favorite buy!I love', 'great compliments']",pos,Bottoms,Pants
4,4,847,47,5,Flattering shirtThis shirt is very flattering ...,General,"['adjustable front tie', 'Flattering shirt', '...",pos,Tops,Blouses
...,...,...,...,...,...,...,...,...,...,...
22623,22623,1104,34,5,Great dress for many occasionsI was very happy...,General Petite,"['great price', 'easy to slip', 'Great dress',...",pos,Dresses,Dresses
22624,22624,862,48,3,Wish it was made of cottonIt reminds me of mat...,General Petite,"['long sleeves', 'soft material', 'nice sleeve...",pos,Tops,Knits
22625,22625,1104,31,3,"Cute, but see throughThis fit well, but the to...",General Petite,"['different fabric', 'fit well', 'never worked...",pos,Dresses,Dresses
22626,22626,1084,28,3,"Very cute dress, perfect for summer parties an...",General,"['much cheaper t', 'cheaper t', 'cute dress', ...",pos,Dresses,Dresses


In [6]:
df2=df.iloc[:,1:4]

In [7]:
df2

,Clothing ID,Age,Rating
0,767,33,4
1,1080,34,5
2,1077,60,3
3,1049,50,5
4,847,47,5
...,...,...,...
22623,1104,34,5
22624,862,48,3
22625,1104,31,3
22626,1084,28,3


In [13]:
df2.describe()

,Clothing ID,Age,Rating
count,22628.000000,22628.000000,22628.000000
mean,919.695908,43.282880,4.183092
std,201.683804,12.328176,1.115911
min,1.000000,18.000000,1.000000
25%,861.000000,34.000000,4.000000
50%,936.000000,41.000000,5.000000
75%,1078.000000,52.000000,5.000000
max,1205.000000,99.000000,5.000000
